## Introduction

In this notebook, we'll delve into the concept of backpropagation in neural networks. Backpropagation is a key algorithm for training neural networks, enabling them to learn from data by adjusting their weights and biases. We'll explore the implementation of backpropagation in the provided neural network code, understand how it computes gradients, and updates the network parameters to minimize the loss function.

Before we proceed, ensure that you have the NumPy library installed. If not, you can install it using the following command:

In [4]:
pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Now to recap, here's our full class of Network from our previous notebook:

In [6]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data):

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                #print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test))
                if (j== epochs-1):
                    accuracy = self.evaluate(test_data)/n_test
                    print("Accuracy: {}".format(accuracy))
                    return accuracy
            
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

Now, let's break down the backpropagation part of this class.

## Content

# 1. Initialization
We'll start by initializing the neural network with random weights and biases. We'll create an instance of the Network class with a specified architecture and random initialization for weights and biases.

# 2. Feedforward Function
Next, we'll review the feedforward method, which performs forward propagation through the network to compute the output. This method takes an input vector and passes it forward through each layer, applying the appropriate weights and biases.

# Code snippet for the feedforward method

# 3. Cost Function
Before diving into backpropagation, we'll define the cost function used to measure the network's performance. We'll use the mean squared error (MSE) as our cost function.

# Code snippet for the cost function (MSE)


# 4. Backpropagation Algorithm
Now, we'll explore the backpropagation algorithm, which computes the gradients of the cost function with respect to the weights and biases of the network. We'll understand how errors are propagated backward through the network to update the parameters.

# Code snippet for the backpropagation algorithm



# 5. Update Rule
Finally, we'll implement the update rule to adjust the weights and biases of the network using the computed gradients and a learning rate.

# Code snippet for the update rule

# 6. Training Process
We'll demonstrate the training process by applying backpropagation to a sample dataset. We'll iterate over the training data, compute gradients using backpropagation, and update the network parameters using stochastic gradient descent (SGD).

# Code snippet for the training process

# 7. Evaluation
After training the network, we'll evaluate its performance on a test dataset to assess its accuracy and generalization.

# Code snippet for evaluating the network

# Conclusion
Backpropagation is a fundamental algorithm for training neural networks, enabling them to learn from data and make accurate predictions. In this notebook, we explored the implementation of backpropagation in the provided neural network code, understanding its role in optimizing network parameters and improving performance.